Dowloading and Isntalling dependencies of spark in the next three coding blocks

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Doing imports required

In [ ]:
!pip install -U imbalanced-learn
!pip install seaborn
from pyspark.sql.functions import col, expr, when
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt 
import matplotlib as mpl
import matplotlib.dates as mdates

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 199 kB 13.2 MB/s 
     |████████████████████████████████| 199 kB 61.8 MB/s 
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.8.1
    Uninstalling imbalanced-learn-0.8.1:
      Successfully uninstalled imbalanced-learn-0.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Reading the contents of the json file, Please paste your link/address here.

In [ ]:
# Please paste the file link here.
DF = spark.read.json('/content/sample.json')

Identifying the Important Attributes for the class labeling.

In [ ]:
imp_Attributes = ['DownloadSpeed','UploadSpeed','Ping','TTFB']

Defining Function to calculate Median. Finding medians of required attributes.

In [ ]:
def calculate_Median(tdf,At):
  tdf = tdf.select(At)
  tdf = tdf.na.drop(subset=[At])
  count = tdf.count()
  tdf = tdf.sort(At)
  if count % 2 ==0:
    Median = (tdf.select(At).collect()[int(count/2)][0] + tdf.select(At).collect()[int(count/2)-1][0])/2
  else:
    Median = tdf.select(At).collect()[int((count-1)/2)][0]
  return Median

Med_DNS = calculate_Median(DF,imp_Attributes[0])
Med_UPS = calculate_Median(DF,imp_Attributes[1])
Med_Ping = calculate_Median(DF,imp_Attributes[2])
Med_TTFB = calculate_Median(DF,imp_Attributes[3])

In [ ]:
print (' Median DNS: ',Med_DNS,"\n",'Median UPS: ',Med_UPS,"\n",'Median Ping: ',Med_Ping,"\n",'Median TTFB: ',Med_TTFB )

 Median DNS:  25.516 
 Median UPS:  5.581 
 Median Ping:  10.0 
 Median TTFB:  806


All the Attributes to be used are defined here in this list.

In [ ]:
Attributes = ['Latitude','Longitude','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
              'AndroidApi','OSVersion','ActiveConnection','WifiBSSID','WifiRssi','WifiLinkSpeed',
              'lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
              'TTFB','fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','Ping',
              'DownloadSpeed','UploadSpeed','LteRssi','WcdmaRssi','GsmRssi']
# Attributes = ['location','@timestamp','Manufacturer','Model','AppName','SdkVersion','AppVersion',
#               'AndroidApi','OSVersion','ActiveConnection','WifiBSSID','WifiRssi','WifiLinkSpeed',
#               'lvl1_name','IP','ASN','ISP','URL','dnsLookup','initialConnection','sslNegotiation',
#               'TTFB','fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','Ping',
#               'DownloadSpeed','UploadSpeed','LteRssi','WcdmaRssi','GsmRssi']

In [ ]:
df = DF.select(Attributes)

Visualizing the Attributes in our data to make sure we are using required ones.

In [ ]:
df.columns

['Latitude',
 'Longitude',
 '@timestamp',
 'Manufacturer',
 'Model',
 'AppName',
 'SdkVersion',
 'AppVersion',
 'AndroidApi',
 'OSVersion',
 'ActiveConnection',
 'WifiBSSID',
 'WifiRssi',
 'WifiLinkSpeed',
 'lvl1_name',
 'IP',
 'ASN',
 'ISP',
 'URL',
 'dnsLookup',
 'initialConnection',
 'sslNegotiation',
 'TTFB',
 'fullyLoaded',
 'PageloadSpeed',
 'loadEventEnd',
 'documentComplete',
 'Ping',
 'DownloadSpeed',
 'UploadSpeed',
 'LteRssi',
 'WcdmaRssi',
 'GsmRssi']

Removing Duplicates from the dataset samples.

In [ ]:
print("Checking and Removing Duplicates\n","Samples before removing:",df.count())
df = df.distinct()
print("Samples after removing Duplicates:",df.count())

Checking and Removing Duplicates
 Samples before removing: 10000
Samples after removing Duplicates: 9865


Defining the Attributes according to the Action required for NULL values.

In [ ]:
Record_Removing_Attributes = ['Latitude','Longitude','@timestamp','AppName','SdkVersion','AppVersion','AndroidApi','OSVersion','ActiveConnection','lvl1_name','IP','ASN','ISP','URL','Manufacturer','Model']
Assign_Zero_Attributes_str = ['WifiBSSID','WifiRssi']
Assign_Zero_Attributes = ['WifiLinkSpeed','dnsLookup','initialConnection','sslNegotiation',
                         'fullyLoaded','PageloadSpeed','loadEventEnd','documentComplete','LteRssi','WcdmaRssi','GsmRssi']

Droping rows of NULL values against record removing attributes and with google URL

In [ ]:
df = df.na.drop(subset=Record_Removing_Attributes)
df = df[df.ActiveConnection != ""] # There were some values in this coulum that were empty but not assigned NULL position.
df=df.where(df.URL !="https://www.google.com.bh")
print("Truncated df count will be :", df.count())

Truncated df count will be : 2214


In [ ]:
print('Before replacing values')
df

Before replacing values


Latitude,Longitude,@timestamp,Manufacturer,Model,AppName,SdkVersion,AppVersion,AndroidApi,OSVersion,ActiveConnection,WifiBSSID,WifiRssi,WifiLinkSpeed,lvl1_name,IP,ASN,ISP,URL,dnsLookup,initialConnection,sslNegotiation,TTFB,fullyLoaded,PageloadSpeed,loadEventEnd,documentComplete,Ping,DownloadSpeed,UploadSpeed,LteRssi,WcdmaRssi,GsmRssi
26.2129,50.58673,2022-01-14T23:20:...,RAVOZ,RAVOZ Z3 Lite,com.speedchecker....,4.2.140-sh,1.0.58,30,11,WIFI,bc:0f:9a:7c:f4:46,-65,26,Capital Governorate,88.201.0.229,5416,Batelco,https://mobile.tw...,0,0,0,499,1590,0.3011183,0,0,10.0,25.516,5.581,0,0,0
26.16837,50.61715,2022-01-15T02:56:...,samsung,SM-A525F,com.speedchecker....,4.2.140-sh,1.0.56-bh,30,11,4G,0,0,0,Capital Governorate,109.161.146.130,31452,Zain Bahrain B.s.c.,https://mobile.tw...,0,0,0,674,5051,1.95153213,0,0,10.0,25.516,5.581,-59,0,0
26.17706,50.60549,2022-01-15T04:13:...,samsung,SM-G930V,com.speedchecker....,4.2.140-sh,1.0.56-bh,24,7.0,WIFI,b2:55:08:88:8d:70,-55,72,Capital Governorate,94.76.8.123,51375,stc Bahrain,https://mobile.tw...,0,0,0,277,1233,0.316082,0,0,10.0,25.516,5.581,0,0,0
26.13697,50.58889,2022-01-15T10:17:...,samsung,SM-J500F,com.trabhr.tracon...,4.2.109,1.1.7,22,5.1.1,WIFI,fc:73:fb:27:3b:ac,-82,9,Southern Governorate,84.255.153.231,5416,Batelco,https://www.faceb...,0,0,0,691,4219,4.490592,4185,4158,10.0,25.516,5.581,0,0,0
26.1535,50.61672,2022-01-15T10:35:...,OPPO,CPH1909,com.speedchecker....,4.2.140-sh,1.0.56-bh,27,8.1.0,4G,0,0,0,Capital Governorate,109.161.208.39,31452,Zain Bahrain B.s.c.,https://www.faceb...,0,0,0,1273,4192,3.672409,4071,4040,10.0,25.516,5.581,0,0,0
26.11502,50.56627,2022-01-15T10:52:...,samsung,SM-N976Q,com.speedchecker....,4.2.140-sh,1.0.56-bh,30,11,WIFI,0,-34,115,Southern Governorate,37.131.125.116,51375,stc Bahrain,https://mobile.tw...,0,0,0,1064,8310,8.730547,8299,8299,10.0,25.516,5.581,0,0,0
26.20181,50.53449,2022-01-15T13:05:...,samsung,SM-J250F,com.speedchecker....,4.2.121,1.0.51-bh,25,7.1.1,WIFI,02:34:b2:82:8c:c1,-22,72,Capital Governorate,109.161.218.98,31452,Zain Bahrain B.s.c.,https://www.insta...,6,245,245,735,3280,108.086975,3228,3221,10.0,25.516,5.581,0,0,0
26.16404,50.55501,2022-01-14T18:38:...,samsung,SM-N9005,uk.co.broadbandsp...,4.2.112,2.6.54,19,4.4.2,WIFI,b6:49:0f:c6:26:a1,-41,72,Southern Governorate,94.76.22.123,51375,stc Bahrain,https://mobile.tw...,57,443,328,849,950,0.316759646,922,922,10.0,25.516,5.581,0,0,0
26.23591,50.57594,2022-01-14T18:50:...,samsung,SM-J415F,com.speedchecker....,4.2.136,1.0.54-bh,28,9,3G,0,0,0,Capital Governorate,37.131.121.236,51375,stc Bahrain,https://www.faceb...,0,0,0,157,2482,0.5448574,0,0,10.0,25.516,5.581,0,0,0
26.26772,50.6559,2022-01-14T20:29:...,HUAWEI,INE-LX1r,com.speedchecker....,4.2.114,1.0.49-bh,28,9,WIFI,18:3d:5e:6b:f3:9a,-61,135,Muharraq Governorate,217.17.241.100,5416,Batelco,https://mobile.tw...,0,0,0,806,0,0.0,0,0,10.0,25.516,5.581,0,0,0


Replacement of NULL values with 0 and Medians accordingly

In [ ]:
df = df.fillna(value = Med_DNS ,subset = imp_Attributes[0])
df = df.fillna(value = Med_UPS ,subset = imp_Attributes[1])
df = df.fillna(value = Med_Ping ,subset = imp_Attributes[2])
df = df.fillna(value = Med_TTFB ,subset = imp_Attributes[3])
df = df.fillna(value=0,subset=Assign_Zero_Attributes)
df = df.fillna(value='0',subset=Assign_Zero_Attributes_str)
print('After replacing values')
df

After replacing values


Latitude,Longitude,@timestamp,Manufacturer,Model,AppName,SdkVersion,AppVersion,AndroidApi,OSVersion,ActiveConnection,WifiBSSID,WifiRssi,WifiLinkSpeed,lvl1_name,IP,ASN,ISP,URL,dnsLookup,initialConnection,sslNegotiation,TTFB,fullyLoaded,PageloadSpeed,loadEventEnd,documentComplete,Ping,DownloadSpeed,UploadSpeed,LteRssi,WcdmaRssi,GsmRssi
26.2129,50.58673,2022-01-14T23:20:...,RAVOZ,RAVOZ Z3 Lite,com.speedchecker....,4.2.140-sh,1.0.58,30,11,WIFI,bc:0f:9a:7c:f4:46,-65,26,Capital Governorate,88.201.0.229,5416,Batelco,https://mobile.tw...,0,0,0,499,1590,0.3011183,0,0,10.0,25.516,5.581,0,0,0
26.16837,50.61715,2022-01-15T02:56:...,samsung,SM-A525F,com.speedchecker....,4.2.140-sh,1.0.56-bh,30,11,4G,0,0,0,Capital Governorate,109.161.146.130,31452,Zain Bahrain B.s.c.,https://mobile.tw...,0,0,0,674,5051,1.95153213,0,0,10.0,25.516,5.581,-59,0,0
26.17706,50.60549,2022-01-15T04:13:...,samsung,SM-G930V,com.speedchecker....,4.2.140-sh,1.0.56-bh,24,7.0,WIFI,b2:55:08:88:8d:70,-55,72,Capital Governorate,94.76.8.123,51375,stc Bahrain,https://mobile.tw...,0,0,0,277,1233,0.316082,0,0,10.0,25.516,5.581,0,0,0
26.13697,50.58889,2022-01-15T10:17:...,samsung,SM-J500F,com.trabhr.tracon...,4.2.109,1.1.7,22,5.1.1,WIFI,fc:73:fb:27:3b:ac,-82,9,Southern Governorate,84.255.153.231,5416,Batelco,https://www.faceb...,0,0,0,691,4219,4.490592,4185,4158,10.0,25.516,5.581,0,0,0
26.1535,50.61672,2022-01-15T10:35:...,OPPO,CPH1909,com.speedchecker....,4.2.140-sh,1.0.56-bh,27,8.1.0,4G,0,0,0,Capital Governorate,109.161.208.39,31452,Zain Bahrain B.s.c.,https://www.faceb...,0,0,0,1273,4192,3.672409,4071,4040,10.0,25.516,5.581,0,0,0
26.11502,50.56627,2022-01-15T10:52:...,samsung,SM-N976Q,com.speedchecker....,4.2.140-sh,1.0.56-bh,30,11,WIFI,0,-34,115,Southern Governorate,37.131.125.116,51375,stc Bahrain,https://mobile.tw...,0,0,0,1064,8310,8.730547,8299,8299,10.0,25.516,5.581,0,0,0
26.20181,50.53449,2022-01-15T13:05:...,samsung,SM-J250F,com.speedchecker....,4.2.121,1.0.51-bh,25,7.1.1,WIFI,02:34:b2:82:8c:c1,-22,72,Capital Governorate,109.161.218.98,31452,Zain Bahrain B.s.c.,https://www.insta...,6,245,245,735,3280,108.086975,3228,3221,10.0,25.516,5.581,0,0,0
26.16404,50.55501,2022-01-14T18:38:...,samsung,SM-N9005,uk.co.broadbandsp...,4.2.112,2.6.54,19,4.4.2,WIFI,b6:49:0f:c6:26:a1,-41,72,Southern Governorate,94.76.22.123,51375,stc Bahrain,https://mobile.tw...,57,443,328,849,950,0.316759646,922,922,10.0,25.516,5.581,0,0,0
26.23591,50.57594,2022-01-14T18:50:...,samsung,SM-J415F,com.speedchecker....,4.2.136,1.0.54-bh,28,9,3G,0,0,0,Capital Governorate,37.131.121.236,51375,stc Bahrain,https://www.faceb...,0,0,0,157,2482,0.5448574,0,0,10.0,25.516,5.581,0,0,0
26.26772,50.6559,2022-01-14T20:29:...,HUAWEI,INE-LX1r,com.speedchecker....,4.2.114,1.0.49-bh,28,9,WIFI,18:3d:5e:6b:f3:9a,-61,135,Muharraq Governorate,217.17.241.100,5416,Batelco,https://mobile.tw...,0,0,0,806,0,0.0,0,0,10.0,25.516,5.581,0,0,0


Counting Acceptable values in data

In [ ]:
df.where((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)).count()

1792

Adding QoE Coulum

In [ ]:
df3 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.UploadSpeed >= 0.5) & (df.Ping <=20) & (df.TTFB <=1200)),1)
                                 .otherwise(0))

Understanding the storage type of data inside the coulums

In [ ]:
df3.printSchema()

root
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- @timestamp: string (nullable = true)
 |-- Manufacturer: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- AppName: string (nullable = true)
 |-- SdkVersion: string (nullable = true)
 |-- AppVersion: string (nullable = true)
 |-- AndroidApi: string (nullable = true)
 |-- OSVersion: string (nullable = true)
 |-- ActiveConnection: string (nullable = true)
 |-- WifiBSSID: string (nullable = false)
 |-- WifiRssi: string (nullable = false)
 |-- WifiLinkSpeed: long (nullable = true)
 |-- lvl1_name: string (nullable = true)
 |-- IP: string (nullable = true)
 |-- ASN: long (nullable = true)
 |-- ISP: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- dnsLookup: long (nullable = true)
 |-- initialConnection: long (nullable = true)
 |-- sslNegotiation: long (nullable = true)
 |-- TTFB: long (nullable = true)
 |-- fullyLoaded: long (nullable = true)
 |-- PageloadSpeed: double (

Extracting name of coulums containing string data

In [ ]:
String_Cols = [item[0] for item in df3.dtypes if item[1].startswith('string')]

In [ ]:
print(String_Cols)

['@timestamp', 'Manufacturer', 'Model', 'AppName', 'SdkVersion', 'AppVersion', 'AndroidApi', 'OSVersion', 'ActiveConnection', 'WifiBSSID', 'WifiRssi', 'lvl1_name', 'IP', 'ISP', 'URL']


'gdf' contains frequency of each type of entry in string data frames.

In [ ]:
gdf = pd.DataFrame()
for x in String_Cols:
  t = df3.groupBy(x).count().toPandas()
  gdf = pd.concat([gdf,t],axis = 1)

Saving Frequency information of string coulums in "String_Frequency.csv" file.

In [ ]:
gdf.to_csv('/content/String_Frequency.csv')

Saving actual data after Cleansing to "After_Cleansing.csv" file

In [ ]:
pdf = df3.toPandas()

In [ ]:
pdf.to_csv('/content/After_Cleansing.csv')

It is saving the description of Data Distribution of all the coulums to "Distribution.csv" file 

In [ ]:
pdf.describe(include='all').to_csv('/content/Distribution.csv')

Code for using ADASYN technique of Oversampling for Data Balancing

In [ ]:
def makeOverSamplesADASYN(X,y):
 #input DataFrame
 #X →Independent Variable in DataFrame\
 #y →dependent Variable in Pandas DataFrame format
 from imblearn.over_sampling import ADASYN 
 sm = ADASYN()
 X, y = sm.fit_resample(X, y)
 return(X,y)

In [ ]:
P_df = df.select(imp_Attributes).toPandas()
Y_df = df3.select('QoE').toPandas()
P_df,Y_df = makeOverSamplesADASYN(P_df,Y_df)

In [ ]:
result = pd.concat([P_df,Y_df])

In [ ]:
result.to_csv('/content/After_ADASYN.csv')

Sample code for multi conditional work on spark

In [ ]:
# df2 = df.withColumn("QoE", when(((df.DownloadSpeed >= 1.5) & (df.DownloadSpeed < 5) & (df.UploadSpeed >= 0.5) & (df.UploadSpeed < 1.5) & (df.Ping >50)& (df.Ping <=100) & (df.TTFB >800)& (df.TTFB <=1200)) ,"A")
#                                  .when((df.DownloadSpeed >= 5) & (df.UploadSpeed >= 1.5) & (df.Ping <=50) & (df.TTFB <=800),"E")
#                                  .otherwise("NA"))

Code for removing Outliers

In [1]:
import numpy as np
from scipy import stats
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # Will remove outliers 3 Standard Deviations from mean

Code for Normalization of values in the data frame

In [ ]:
# copy the data
df_min_max_scaled = df.copy()
  
# apply normalization techniques
for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())  